In [53]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emojiprediction/train_emoji.csv
/kaggle/input/emojiprediction/test_emoji.csv
/kaggle/input/glove6b50dtxt/glove.6B.50d.txt


In [54]:
# !pip install emoji

In [55]:
# import emoji

In [56]:
# emoji.EMOJI_UNICODE

In [57]:
# print(emoji.emojize('Python is :thumbs_up:'))

In [58]:
# print(emoji.demojize('Python is 👍'))

In [59]:
# print(emoji.emojize("Python is fun :flexed_biceps:"))

In [60]:
# print("\u2764\uFE0F")

In [61]:
# emoji_dictionary={
#     "0":"\u2764\uFE0F",
#     "1":":baseball:",
#     "2":":grinning_face_with_big_eyes:",
#     "3":":disappointed_face:",
#     "4":":fork_and_knife:"
# }

In [62]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from keras.utils import np_utils

import matplotlib.pyplot as plt

In [63]:
train=pd.read_csv('/kaggle/input/emojiprediction/train_emoji.csv',header=None)
test=pd.read_csv('/kaggle/input/emojiprediction/test_emoji.csv',header=None)

In [64]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [65]:
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [66]:
emoji_dict = { 
              0 : ":heart:", 
              1 : ":baseball:", 
              2 : ":smile:", 
              3 : ":disappointed:", 
              4 : ":fork_and_knife:"
             }

In [67]:
for ix in emoji_dict.keys():
    print (ix,end=" ")
    print (emoji.emojize(emoji_dict[ix], use_aliases=True))

0 ❤
1 ⚾
2 😄
3 😞
4 🍴


In [68]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print (X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
print ("-------------------------")
print (X_train[0],Y_train[0])


(132,) (132,) (56,) (56,)
-------------------------
never talk to me again 3


In [69]:
for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()

# Splitting the test data from sentences to words
for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()
    
# Converting labels into categorical form
Y_train = np_utils.to_categorical(Y_train)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [70]:
X_train[0]

['never', 'talk', 'to', 'me', 'again']

In [71]:
np.unique(np.array([len(ix) for ix in X_train]) , return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 4,  5, 26, 35, 20, 21, 11,  5,  1,  4]))

In [72]:
np.unique(np.array([len(ix) for ix in X_test]) , return_counts=True)

(array([2, 3, 4, 5, 6, 7, 8]), array([ 3, 12, 16, 17,  3,  4,  1]))

In [73]:
embeddings_index = {}

f = open('/kaggle/input/glove6b50dtxt/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [74]:
embeddings_index['happy'].shape

(50,)

In [75]:
from scipy import spatial
# Checking cosine similarity of words happy and sad
spatial.distance.cosine(embeddings_index["happy"], embeddings_index["sad"])

0.31093674898147583

In [76]:
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

In [77]:
for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embedding_matrix_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
        
for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        embedding_matrix_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]

In [78]:
print (embedding_matrix_train.shape, embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


In [84]:
model = Sequential()
model.add(LSTM(128, input_shape=(10,50), return_sequences=True))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 10, 128)           91648     
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
Total params: 223,877
Trainable params: 223,877
Non-trainable params: 0
_________________________________________________________________


In [85]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [86]:
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
5/5 [==============================] - 0s 16ms/step - loss: 1.5712 - accuracy: 0.2576
Epoch 2/50
5/5 [==============================] - 0s 16ms/step - loss: 1.5138 - accuracy: 0.2879
Epoch 3/50
5/5 [==============================] - 0s 17ms/step - loss: 1.4245 - accuracy: 0.3485
Epoch 4/50
5/5 [==============================] - 0s 14ms/step - loss: 1.3467 - accuracy: 0.5682
Epoch 5/50
5/5 [==============================] - 0s 14ms/step - loss: 1.2097 - accuracy: 0.6212
Epoch 6/50
5/5 [==============================] - 0s 15ms/step - loss: 1.1077 - accuracy: 0.6667
Epoch 7/50
5/5 [==============================] - 0s 14ms/step - loss: 0.9378 - accuracy: 0.6591
Epoch 8/50
5/5 [==============================] - 0s 14ms/step - loss: 0.8117 - accuracy: 0.6894
Epoch 9/50
5/5 [==============================] - 0s 15ms/step - loss: 0.7316 - accuracy: 0.7348
Epoch 10/50
5/5 [==============================] - 0s 15ms/step - loss: 0.6016 - accuracy: 0.7727
Epoch 11/50
5/5 [============

In [87]:
pred=model.predict_classes(embedding_matrix_test)

In [89]:
float(sum(pred==Y_test))/pred.shape[0]

0.625

In [91]:
for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] == Y_test[ix]:
        print(ix)
        print (test[0][ix],end=" ")
        print (emoji.emojize(emoji_dict[pred[ix]], use_aliases=True),end=" ")
        print (emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True))

0
['I', 'want', 'to', 'eat'] 🍴 🍴
1
['he', 'did', 'not', 'answer'] 😞 😞
2
['he', 'got', 'a', 'raise'] 😄 😄
4
['ha', 'ha', 'ha', 'it', 'was', 'so', 'funny'] 😄 😄
8
['where', 'is', 'the', 'food'] 🍴 🍴
9
['Stop', 'making', 'this', 'joke', 'ha', 'ha', 'ha'] 😄 😄
10
['where', 'is', 'the', 'ball'] ⚾ ⚾
14
['Let', 'us', 'go', 'play', 'baseball'] ⚾ ⚾
15
['This', 'stupid', 'grader', 'is', 'not', 'working'] 😞 😞
17
['Congratulation', 'for', 'having', 'a', 'baby'] 😄 😄
18
['stop', 'messing', 'around'] 😞 😞
20
['I', 'love', 'taking', 'breaks'] ❤ ❤
22
['I', 'boiled', 'rice'] 🍴 🍴
24
['Why', 'are', 'you', 'feeling', 'bad'] 😞 😞
25
['I', 'am', 'upset'] 😞 😞
27
['My', 'grandmother', 'is', 'the', 'love', 'of', 'my', 'life'] ❤ ❤
30
['I', 'miss', 'you', 'so', 'much'] ❤ ❤
31
['throw', 'the', 'ball'] ⚾ ⚾
32
['My', 'life', 'is', 'so', 'boring'] 😞 😞
33
['she', 'said', 'yes'] 😄 😄
34
['will', 'you', 'be', 'my', 'valentine'] ❤ ❤
35
['he', 'can', 'pitch', 'really', 'well'] ⚾ ⚾
36
['dance', 'with', 'me'] 😄 😄
37
['I', 'am', 's